The iexfinance API seems to be not working. For now, this example does not work.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np; np.random.seed(1)
import matplotlib.pyplot as plt
import pandas as pd
from extquadcontrol import dp_finite, dp_infinite, ExtendedQuadratic, \
    FiniteHorizonSystem, InfiniteHorizonSystem, AffinePolicy, Policy, TimeInvariantAffinePolicy
from scipy.linalg import block_diag
import cvxpy as cvx

from iexfinance import get_historical_data
from datetime import datetime
import pandas as pd

In [ ]:
def extended_quadratic_to_cvx(f, x):
    if f.F.shape[0] == 0:
        return .5 * cvx.quad_form(x, f.P) + f.q * x + .5 * f.r, []
    else:
        return .5 * cvx.quad_form(x, f.P) + f.q * x + .5 * f.r, [f.F * x + f.g == 0]

In [ ]:
# import quandl;
# quandl.ApiConfig.api_key = "INSERT API KEY HERE"
tickers = ['FXI','GDX','IWM','QQQ','SPY','XLF','XOP']
n = len(tickers)
P = []
for t in tickers:
    df = get_historical_data(t, output_format='pandas')
    P.append(np.array(df.open)[:,None])
prices = np.concatenate(P,axis=1) 

In [ ]:
vix = np.array(quandl.get('CHRIS/CBOE_VX5',start_date='2011-01-01').Open)[-prices.shape[0]:]

In [ ]:
vix = vix[:-1]
prices = prices[1:]

In [ ]:
i = 0
for p in prices.T:
    plt.plot(p,label=tickers[i])
    i += 1
plt.legend()

In [ ]:
plt.plot(vix)

In [ ]:
dvix = np.diff(vix)/vix[:-1]
dvix = dvix[:-1]
plt.plot(dvix)

In [ ]:
plt.hist(dvix, bins=50);

In [ ]:
ranges = np.r_[
    -np.logspace(-2.5,np.log10(-np.min(dvix)+1e-4),3)[::-1],
    np.logspace(-2.5,np.log10(np.max(dvix)+1e-4),3)
]
np.set_printoptions(precision=3)
ranges

In [ ]:
cut = pd.cut(dvix,ranges,labels=np.arange(len(ranges)-1))
cut.value_counts()

In [ ]:
K = len(cut.value_counts())
Pi = 0*np.ones((K,K))
for i in range(len(cut)-1):
    Pi[cut[i+1],cut[i]] += 1

In [ ]:
Pi/=np.sum(Pi,axis=0)

In [ ]:
plt.imshow(Pi)
Pi
# Pi = np.eye(K)

In [ ]:
returnslog = np.log(1+np.diff(prices,axis=0)/prices[:-1])
returnslog = returnslog[1:]
mus = []
sigmas = []
means = []
covs = []
for i in range(K):
    mu = np.mean(returnslog[cut==i],axis=0)
    sigma = np.cov(returnslog[cut==i].T) + 1e-10*np.eye(n)
    mus.append(mu)
    sigmas.append(sigma)
    
    mean = np.exp(mu + .5*np.diag(sigma))
    covariance = np.diag(mean)@(np.exp(sigma)-np.ones((n,n)))@np.diag(mean)
    means.append(mean)
    covs.append(covariance)

In [ ]:
means

In [ ]:
np.set_printoptions(precision=5,suppress=True)
plt.figure(figsize=(14,10))
for i in range(n):
    plt.plot([m[i] for m in means], c='black')

In [ ]:
plt.figure(figsize=(14,10))
for i in range(n):
    plt.plot([np.sqrt(c[i,i]) for c in covs], c='black')

In [ ]:
means[4]

In [ ]:
covs[4]

In [ ]:
w = cvx.Variable(n)
ws = []
for gamma in np.logspace(-1,5,100):
    objective = -means[2]*w + gamma/2*cvx.quad_form(w,covs[2])
    constraints = [np.ones(n)*w == 1, w >=0]

    problem = cvx.Problem(cvx.Minimize(objective),constraints)
    result = problem.solve(solver='OSQP')
    ws.append(np.array(w.value)[:,None])
ws = np.concatenate(ws,axis=1)
for i in range(n):
    plt.semilogx(np.logspace(-1,5,100),ws[i], label=tickers[i])
plt.legend()

In [ ]:
b = [prices[-1]*1e-7 for _ in range(K)]
gamma = 1e-1
T = 30
N = 30

def sample(t,N):
    As = np.zeros((N,K,n,n))
    Bs = np.zeros((N,K,n,n))
    cs = np.zeros((N,K,n))
    for s in range(K):
        mu,sigma=mus[s],sigmas[s]
        r = np.exp(np.random.multivariate_normal(mu,sigma,size=N))
        for i in range(n):
            As[:,s,i,i] = r[:,i]
            Bs[:,s,i,i] = r[:,i]
    gs = []
    for s in range(K):
        mean,cov = means[s],covs[s]
        g = ExtendedQuadratic(np.c_[np.r_[gamma*cov,gamma*cov],np.r_[gamma*cov,gamma*cov+np.diag(b[s])]],-np.r_[mean,mean],0,np.c_[np.zeros((1,n)),np.ones((1,n))],np.zeros(1))
        gs.append(g)
    return As,Bs,cs,[gs for _ in range(N)],Pi
sample_infinite = lambda N: sample(0,N)
g_T = [ExtendedQuadratic(np.zeros((n,n)),np.zeros(n),0) for s in range(K)]

In [ ]:
Vs, Qs, policies = dp_finite(sample,g_T,T,N)
V = Vs[0]
Q = Qs[0]
policy = policies[0]

In [ ]:
Kpolicy,kpolicy = policy[0]
# Kpolicy

In [ ]:
xstar = -np.linalg.lstsq(Kpolicy,kpolicy,rcond=None)[0]
np.allclose(-Kpolicy@xstar,kpolicy)

In [ ]:
np.set_printoptions(precision=3,suppress=False)
for i in range(n):
    print (tickers[i], (xstar/np.sum(xstar))[i])

In [ ]:
np.linalg.eigvals(Kpolicy)

In [ ]:
system = InfiniteHorizonSystem(sample_infinite,K)

In [ ]:
final_wealths = []
np.random.seed(1)
for i in range(100):
    Xs, Us, Modes, cost = system.simulate(1000*np.ones(n),3,30,TimeInvariantAffinePolicy(policy)) 
    if i <= 15:
        plt.plot(np.sum(Xs,axis=1),color='black')
    final_wealths.append(np.sum(Xs,axis=1)[-1])
plt.ylabel('total value')
plt.xlabel('t')
plt.savefig('figs/portfolio1.pdf')

In [ ]:
Xs.shape

In [ ]:
# Xs/np.sum(Xs,axis=1)[:,None]

In [ ]:
i = 0
for x in (Xs/Xs.sum(axis=1)[:,None]).T:
    plt.plot(x, label=tickers[i])
    i += 1
plt.legend()
plt.ylabel('allocation')
plt.xlabel('t')
plt.savefig('figs/portfolio2.pdf')

In [ ]:
plt.hist(np.array(final_wealths)/(1000*n)-1.,bins=30,color='black',density=True);
plt.xlim(-.125,.125)
plt.ylabel('count')
plt.xlabel('returns')
plt.savefig('figs/portfolio3.pdf')

In [ ]:
100*(np.mean(np.array(final_wealths)/(1000*n))-1)

In [ ]:
np.std(100*np.array(final_wealths)/(1000*n))

In [ ]:
plt.step(np.arange(len(Modes)),np.array(Modes)+1, color='black')
plt.ylabel('mode')
plt.xlabel('t')
plt.savefig('figs/portfolio4.pdf')

In [ ]:
# plt.plot(np.sum(np.abs(Us),axis=1),color='black')
# plt.ylabel('trading volume')
# plt.xlabel('t')
# plt.savefig('figs/portfolio3.pdf')

In [ ]:
class LongOnlyPolicy(Policy):
    def __init__(self, Q):
        self.Q = Q

    def __call__(self, t, x, s):
        Q = self.Q[s]
        Q = Q.affine_composition(np.r_[np.zeros((n,n)),np.eye(n)],np.r_[x,np.zeros(n)])
        u = cvx.Variable(n)
        Q, constraints = extended_quadratic_to_cvx(Q, u)
        problem = cvx.Problem(cvx.Minimize(Q), constraints+[x+u>=1e-3])
        result = problem.solve(solver='OSQP')
        return np.array(u.value).flatten()

In [ ]:
np.random.seed(1)
final_wealths = []
for _ in range(20):
    Xs, Us, Modes, cost = system.simulate(1000*np.ones(n),3,T,LongOnlyPolicy(Q)) 
    plt.plot(np.sum(Xs,axis=1),color='black')
    final_wealths.append(np.sum(Xs,axis=1)[-1])
plt.savefig('figs/portfolio1-longonly.pdf')

In [ ]:
np.mean(np.array(final_wealths)/(1000*n))

In [ ]:
np.std(np.array(final_wealths)/(1000*n))

In [ ]:
i = 0
for x in (Xs/Xs.sum(axis=1)[:,None]).T:
    plt.plot(x, label=tickers[i])
    i += 1
plt.legend()
plt.savefig('figs/portfolio2-longonly.pdf')

In [ ]:
plt.step(np.arange(len(Modes)),Modes, color='black')
plt.savefig('figs/portfolio4-longonly.pdf')

In [ ]:
plt.plot(np.sum(np.abs(Us),axis=1),color='black')
plt.savefig('figs/portfolio3-longonly.pdf')